In [1]:
import argparse
import math
import sys
import time
import copy

In [2]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization, regularizers
from keras.layers.noise import GaussianNoise
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical

In [3]:
K.image_data_format()

'channels_last'

In [4]:
K.set_image_dim_ordering('th')

In [5]:
K.image_data_format()

'channels_first'

In [6]:
import tensorflow as tf
from keras.backend import tensorflow_backend

In [7]:
config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth = True))
session = tf.Session(config = config)
tensorflow_backend.set_session(session)

In [8]:
import os
import pandas as pd
import numpy as np

In [16]:
import PIL.Image as pilimg

In [9]:
model_path = './fm_cnn_BN.h5'

In [10]:
callbacks = [
    EarlyStopping(monitor='val_acc', patience=10, mode = 'max', verbose=1),
    ModelCheckpoint(model_path, monitor='val_acc', save_weights_only=True, mode='max', verbose = 0)
]

In [11]:
data_dir = '../../data/plant_seed/plant-seeding-classification/'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir,'test')
sample_submission = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))

In [12]:
CATEGORIES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']

In [13]:
SEED = 42

In [14]:
for category in CATEGORIES:
    print('{} {} images'.format(category, len(os.listdir(os.path.join(train_dir, category)))))

Black-grass 263 images
Charlock 390 images
Cleavers 287 images
Common Chickweed 611 images
Common wheat 221 images
Fat Hen 475 images
Loose Silky-bent 654 images
Maize 221 images
Scentless Mayweed 516 images
Shepherds Purse 231 images
Small-flowered Cranesbill 496 images
Sugar beet 385 images


In [17]:
train =[]
for category_id, category in enumerate(CATEGORIES):
    for file in os.listdir(os.path.join(train_dir, category)):
        file_name = 'train/{}/{}'.format(category, file)
        file_location = os.path.join(data_dir, file_name)
        im = pilimg.open(file_location)
        pix = np.array(im)
        height, width = pix.shape[:2]
        
        train.append(['train/{}/{}'.format(category, file), category_id, category, height, width])
train = pd.DataFrame(train, columns=['file', 'category_id', 'category', 'height', 'width'])

train.shape

(4750, 5)